In [41]:
import csv
import cv2
import matplotlib.pyplot as plt
from plotnine import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
%matplotlib inline
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [42]:
class MuraDataset(torch.utils.data.Dataset):
    
    def __init__(self,text_file,root_dir, transform = transforms.Compose([transforms.Resize((224,224)),
                                                                          transforms.ToTensor(),
                                                                          transforms.Normalize(
                                                                              mean=[0.456],
                                                                              std= [0.225])])): #Normalize to pretrained VGGNet
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
        #File with the Path
        self.name_frame = pd.read_csv(text_file,sep=",",usecols=[0],dtype = 'str',nrows = 1000)
        #File with labels
        self.label_frame = pd.read_csv(text_file,sep=",", usecols=[1], nrows = 1000)
        self.root_dir = root_dir
        self.transform = transform
                                       
    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        #Pull image
        img_name = os.path.join(self.root_dir, self.name_frame.iloc[idx, 0])
        
        #Apply adapative threshold to highlight key features in the image   
        image = cv2.imread(img_name,0)
        image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)  
        image = Image.fromarray(image)
        image = image.convert('L')
        
        #Apply tensor transformations to images
        image = self.transform(image) 
        
        # I included labels in the train_image_path file in excel using a find("positive")
        labels = self.label_frame.iloc[idx,0] 
        
        #Formatting for labels
        labels = np.array(labels)
        labels = np.reshape(labels, (1,1))
        labels= torch.from_numpy(labels.astype('int'))

        sample = {'image': image, 'labels': labels}
        
        return sample

In [43]:
MuraTrainSet = MuraDataset(text_file ='/Users/JosephVele/MURA-v1.1/Shoulder_Mura.csv',
                                   root_dir = '/Users/JosephVele')

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=100,shuffle=True, num_workers=0)

MuraTestSet = MuraDataset(text_file ='/Users/JosephVele/MURA-v1.1/Shoulder_Mura_test.csv',
                                   root_dir = '/Users/JosephVele')

MuraTestLoader = torch.utils.data.DataLoader(MuraTestSet,batch_size=30,shuffle=False, num_workers=0)

In [49]:
#Calculate mean &std - batch was sent to 8379 to match number of records in one batch

for i_batch,sample_batched in enumerate(MuraTrainLoader,0):
    numpy_image = sample_batched['image'].numpy()
    image_mean = np.mean(numpy_image, axis=(0,2,3))
    image_std = np.std(numpy_image, axis=(0,2,3))


#image_mean=np.mean(x, axis= (0))
#image_std=np.std(x, axis=(0))

In [50]:
#Confirm images have been normalized appropriately with 0 mean and 1 std
#This will help with vanishing gradient problem
print(image_mean)
print(image_std)

[0.02527229]
[0.96729124]


In [2]:
import torchvision.models as models
dense_net169 = models.densenet169(pretrained=True)

/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /Users/JosephVele/.torch/models/densenet169-b2777c0a.pth
100.0%


In [32]:
for param in dense_net169.features.parameters():
    param.requires_grad = False
#Adjust to grayscale = 1 channel. Requires gradient is True by default
dense_net169.features._modules['conv0']  = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
dense_net169.classifier = nn.Linear(in_features=1664, out_features=1, bias=True)
#dense_net169

for param in dense_net169.classifier.parameters():
    print(param.size())


torch.Size([1, 1664])
torch.Size([1])


In [44]:
# Define hyper-parameters
#******************************#

learning = 1e-3 # Learning Rate 
moment =.9 #Momentum 
w = 1e-2
b = [.9,999]
#******************************#

In [45]:
# 4.1 Define criterion and optimizer
#************************************#
import torch.optim as optim


criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(dense_net169.classifier.parameters(), lr=learning, weight_decay = w)

#************************************#

# 4.2 Train the model
# 4.3 Please store and print training and validation loss&accuracy after each epoch
#********************************************#

train_losses = []
test_losses = []
acc_data_train=[]
acc_data_test=[]


def train(epoch):
    dense_net169.train()
    train_loss = 0
    correct = 0
    for i, sample_batched in enumerate(MuraTrainLoader,1):
        inputs = sample_batched['image']
        labels = sample_batched['labels']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = dense_net169(inputs)
        labels = labels.view(len(sample_batched['labels']),-1)
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        train_loss+= loss.item()
        
        pred = (torch.sigmoid(outputs).data > 0.5).int()
        labels = labels.int()
        correct += pred.eq(labels.data.view_as(pred)).sum()


    
    #Print Results from first Epoch
    #Accuracy
    acc =  100.00 * float(correct) / float(len(MuraTrainLoader.dataset))
    acc_data_train.append(acc)
    #Average Loss
    train_loss=float(train_loss)/float(i)
    train_losses.append(train_loss)
    # print statistics        
    print('Train Epoch:{} Loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
            train_loss, correct, len(MuraTrainLoader.dataset), acc))

def test(epoch):
    #Have our model in evaluation mode
    dense_net169.eval()
    #Set losses and Correct labels to zero
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(MuraTestLoader,1):
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            outputs = dense_net169(inputs)
            labels = labels.view(len(sample_batched['labels']),-1)
            loss = criterion(outputs.float(), labels.float())
            test_loss += loss.item()

            pred = (torch.sigmoid(outputs).data > 0.5).int()
            labels = labels.int()
            correct += pred.eq(labels.data.view_as(pred)).sum()

        test_loss = test_loss/i
        test_losses.append(test_loss)
        
        acc =  100.00 * float(correct) / float(len(MuraTestLoader.dataset))
        acc_data_test.append(acc)
        print('Test Epoch:{} Loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
            test_loss, correct, len(MuraTestLoader.dataset),
            acc))

In [ ]:
#Train + Test per epoch
for epoch in range(1,10): 
    train(epoch)
    test(epoch)

Train Epoch:1 Loss: 0.6220, Accuracy: 662/1000 (66.20%)

Test Epoch:1 Loss: 0.6629, Accuracy: 347/562 (61.74%)

Train Epoch:2 Loss: 0.6089, Accuracy: 697/1000 (69.70%)

Test Epoch:2 Loss: 0.6609, Accuracy: 344/562 (61.21%)

Train Epoch:3 Loss: 0.6054, Accuracy: 696/1000 (69.60%)

Test Epoch:3 Loss: 0.6671, Accuracy: 340/562 (60.50%)

Train Epoch:4 Loss: 0.5878, Accuracy: 704/1000 (70.40%)

Test Epoch:4 Loss: 0.6670, Accuracy: 332/562 (59.07%)

Train Epoch:5 Loss: 0.5859, Accuracy: 710/1000 (71.00%)

Test Epoch:5 Loss: 0.6675, Accuracy: 333/562 (59.25%)

Train Epoch:6 Loss: 0.5757, Accuracy: 729/1000 (72.90%)

Test Epoch:6 Loss: 0.6538, Accuracy: 354/562 (62.99%)

Train Epoch:7 Loss: 0.5683, Accuracy: 744/1000 (74.40%)

Test Epoch:7 Loss: 0.6575, Accuracy: 351/562 (62.46%)

Train Epoch:8 Loss: 0.5651, Accuracy: 736/1000 (73.60%)

Test Epoch:8 Loss: 0.6579, Accuracy: 353/562 (62.81%)

Train Epoch:9 Loss: 0.5553, Accuracy: 740/1000 (74.00%)

Test Epoch:9 Loss: 0.6541, Accuracy: 360/562 (6

In [5]:
#Not Used - Originally constructing from scratch
#import torch.nn.functional as F

#class ConvNet(nn.Module):
#    def __init__(self, num_classes=2, drop1 = .1, drop2 = .5):
#        super(ConvNet, self).__init__()
#        # 3.1 Initialization
#        #******************************#
#        # 1 input image channel, 10 output channels, 1x1 square convolution
#        # kernel
#        self.conv1 = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)        
#        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2) 
#        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)  
#        self.conv4 =  nn.Conv2d(384, 256, kernel_size=3, padding=1)
#        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
#              
#        self.dense1_drop = nn.Dropout()   
#        self.dense1 = nn.Linear(256 * 6 * 6, 4096)
#        self.dense2_drop = nn.Dropout()
#        self.dense2 = nn.Linear(4096, 4096)
#        self.dense3 = nn.Linear(4096, num_classes-1)
#                
        
        #******************************#
#    def forward(self, x):
        # 3.2 Define Neural Network
        #******************************#
        
        #convolutional layers
#        x = F.max_pool2d(F.relu(self.conv1(x), inplace=True),kernel_size=3, stride=2 )
#        x = F.max_pool2d(F.relu(self.conv2(x), inplace=True),kernel_size=3, stride=2 )
#        x = F.relu(self.conv3(x), inplace=True)
#        x = F.relu(self.conv4(x), inplace=True)
#        x = F.max_pool2d(F.relu(self.conv5(x), inplace=True),kernel_size=3, stride=2 )
        
        #flatten
#        x = x.view(x.size(0), 256 * 6 * 6)
        
        #dense layers
#        x = F.relu(self.dense1(self.dense1_drop(x)))
#        x = F.relu(self.dense2(self.dense2_drop(x)))
        
#        x = self.dense3(x)
        
        #******************************#
#        return x
        
                       
#model = ConvNet(num_classes=2).to(device)
# Note: what is the difference between 'same' and 'valid' padding? 
# Take a look at the outputs to understand the difference.

In [33]:
#Tested initially but didn't work
#for param in vgg_19_bn.parameters():
#    param.requires_grad = False

#vgg_19_bn.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#vgg_19_bn.classifier._modules['7'] = vgg_19_bn.classifier._modules['4']
#vgg_19_bn.classifier._modules['8'] = nn.Dropout(p=0.5)
#vgg_19_bn.classifier._modules['9'] = nn.Linear(1000, 1)

